In this workbook, I assume that I have cleaned the data for 2017 and for 2014-15-16. Instead of splitting the data into training and testing set and may suffer from class imbalances, I decide to use data from 2014 through 2016 as the training set and data for 2017 as the testing set. If this approach is successful, I can validate it by further backtesting:
- Train data on 2014, test on 2015
- Train data on 2014-15, train on 2016

In [1]:
import os 
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics

from sklearn.externals import joblib
salary_limit = 7e4

os.chdir(r"C:\Users\nkieu\Desktop\Python\Loan data")
accepted2017 = pd.read_csv(r".\input\accepted2017_clean.csv", encoding = "ISO-8859-1")
accepted_train = pd.read_csv(r".\input\acceptedtrain_clean.csv", encoding = "ISO-8859-1")
# accepted_train_copy = accepted_train.copy()

In [2]:
print("Training size", accepted_train[accepted_train.loan_status != 'Current'].shape)
print("Testing size", accepted2017[accepted2017.loan_status != 'Current'].shape)

Training size (508676, 96)
Testing size (33609, 79)


The cleaned dataset are created using the following functions:

In [2]:
def getAUC(model, X_train, y_train, X_test, y_test):
    res = [0.0, 0.0]
    y_train_score = model.predict_proba(X_train)[:, 1]
    res[0] = metrics.roc_auc_score(y_train, y_train_score)
    print("In sample", res[0])
    
    y_test_score = model.predict_proba(X_test)[:, 1]
    res[1] = metrics.roc_auc_score(y_test, y_test_score)
    print("In sample", res[1])
    
    return res

def GetAccepted2017():
    accepted_original = pd.read_csv('accepted2017.csv', encoding = "ISO-8859-1")
    
    accepted = accepted_original.drop(['title', 'funded_amnt'], axis = 1)
    # accepted = accepted.drop(['title', 'funded_amnt', 'funded_amnt_inv'], axis = 1)
    
    # linear combination of other columns
    accepted = accepted.drop(['out_prncp_inv','total_rec_prncp','total_pymnt_inv'], axis = 1)
    
    # repeated information
    accepted = accepted.drop(["fico_range_low", "last_fico_range_low"], axis = 1)
    accepted = accepted.drop(['addr_state', 'initial_list_status', 'pymnt_plan'], axis = 1)
    accepted = accepted.drop(['application_type', 'hardship_flag', 'disbursement_method','debt_settlement_flag','sub_grade'], axis = 1)
    accepted = accepted.drop(['zip_code', 'id','policy_code','tax_liens', 'tax_liens'], axis = 1)
    
    ##############
    #
    # Beginning transformation
    #
    ##############
    accepted.issue_d = pd.to_datetime(accepted.issue_d)
    accepted.earliest_cr_line = pd.to_datetime(accepted.earliest_cr_line)
    
    accepted['issue_yr'] = accepted.issue_d.dt.year
    
    accepted['earliest_cr_yr'] = pd.to_datetime(accepted['earliest_cr_line']).dt.year
    accepted = accepted.drop('earliest_cr_line', axis = 1)
    

    accepted.term = accepted.term.apply(str)
    accepted['term'] = accepted['term'].apply(lambda x: x.strip().split(" ")[0])
    
    ### Later: we do not even use emp_length
    accepted.emp_length[accepted.emp_length == '10+ years'] = '10 years'
    accepted.emp_length[accepted.emp_length == '< 1 year'] = '0 year'
    accepted.emp_length = accepted.emp_length.apply(lambda x: int(x.split(' ')[0]))
    
    accepted.emp_title = accepted.emp_title.apply(lambda x: x.strip().lower())
    
    accepted.emp_title = accepted.emp_title.replace(['registered nurse', 'rn'], 'nurse')
    accepted.home_ownership = accepted.home_ownership.replace(['ANY', 'NONE'], 'RENT')
    accepted = accepted.drop("avg_cur_bal", axis = 1)
    
    accepted['target'] = 1
    accepted.loc[(accepted.loan_status == 'Current') |(accepted.loan_status == 'Fully Paid'),
          'target'] = 0
    
    leq1mil = accepted['annual_inc'] <= 1e6
    accepted = accepted[leq1mil]
    accepted = accepted[accepted.dti < 100.0]
    
    return accepted

def GetLog(dataframe, header, drop = True):
    if header not in dataframe.columns.values:
        print(header, "not found in data frame")
        return dataframe
    
    dataframe.loc[:, "log_"+header] = np.log(dataframe.loc[:, header])
    if drop:
        return dataframe.drop([header], axis = 1)
    else:
        return dataframe

def Transform(accepted, drop = False):
    accepted1 = GetLog(accepted, 'annual_inc', drop)
    accepted1 = GetLog(accepted1, 'installment', drop)
    
    accepted1['disposable_inc'] = (1 - accepted1['dti'])*accepted1['annual_inc']/12 - accepted1['installment']
    accepted1['install_loan_ratio'] = accepted1['installment'] / accepted1['loan_amnt']
    accepted1['dti_new']= accepted1['disposable_inc']*12/ accepted1['annual_inc']
    
    return accepted1
    
def GetXY(train, features):
    # To leave behind Fully paid / Default
    train = train.loc[(train.loan_status != 'Current') | (train.loan_status != 'In Grace Period'),
                      features]

    categorical_features = ['term', 'home_ownership', 'verification_status']
    
    for cat_feature in categorical_features:
        if cat_feature in train.columns:
            train = pd.get_dummies(train, prefix = [cat_feature], columns = [cat_feature], drop_first = True)
            
    X = train.drop(['loan_status', 'target'], axis = 1)
    y = train.target
    
    return X, y

Let's explore our testing data to see the rough relationship between variables.

In [3]:
accepted2017.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'purpose', 'dti', 'delinq_2yrs',
       'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'out_prncp', 'total_pymnt', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'last_fico_range_high', 'collections_12_mths_ex_med', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'num_accts_ever_120_pd',
       'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl',
     

Generally, *bc* stands for bank cards. *il* stands for installment.
*num_actv_bc_tl* is number of active bank card accounts.

I calculate a few more variables:
- *Balance_annual_inc* is the ratio of current balance over annual income, similarly for *install_annual*.
- *install_loan_ratio* is the ratio of monthly installment / loan amount. This should be very similar to rate, and therefore to the grade of the loan
- *disposable_inc* is the amount of disposable income each month after mortgage payment and payment for Lending Club loans
- *dti_new* is *disposable_inc* \* 12 / *annual_inc*

I plan to approach using a series of tree-model, which does not handle well the difference and interaction between features. 

There are a number of variables that may be considered look-ahead here, which are of no use in practice. I manually filtered for these variables and come up a list of potential variables:

In [3]:
selected_feature = ['loan_amnt', 'term', 'int_rate', 'installment'
            # , 'grade' - grade is dropped because it is highly correlated with int_rate
            , 'emp_length', 'home_ownership', 'annual_inc', 'verification_status'
            # ,'issue_d'
            # , 'purpose'
            ,'dti', 'delinq_2yrs'
            ,'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal'
            ,'revol_util', 'total_acc'
            #, 'out_prncp', 'total_pymnt', 'total_rec_int','total_rec_late_fee', 'recoveries', 'collection_recovery_fee'
            #,'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
            #,'last_fico_range_high', 'collections_12_mths_ex_med', 'acc_now_delinq'
            #,'tot_coll_amt', 
            ,'tot_cur_bal', 'total_rev_hi_lim','acc_open_past_24mths', 'bc_open_to_buy', 'bc_util'
            ,'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct'
            ,'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl'
            ,'mort_acc', 'mths_since_recent_bc', 'num_accts_ever_120_pd'
            ,'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl'
            ,'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0'
            ,'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m'
            ,'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75'
            ,'pub_rec_bankruptcies', 'tot_hi_cred_lim'
            ,'total_bal_ex_mort'
            ,'total_bc_limit', 'total_il_high_credit_limit'
            #,'issue_yr'
            #,'earliest_cr_yr'
            ,'loan_status','target'
            ,'cr_yr_before_loan'
            ,'log_annual_inc', 'log_installment'
            ,'disposable_inc', 'install_loan_ratio', 'dti_new', 'balance_annual_inc'
            ,'install_annual']

In [4]:
def Transform_2(accepted_train):
    accepted_train = accepted_train.loc[accepted_train.loan_status != 'Current']
    accepted_train['disposable_inc'] = (1 - accepted_train['dti'])*accepted_train['annual_inc']/12 - accepted_train['installment']
    accepted_train['dti_new']= accepted_train['disposable_inc']*12/ accepted_train['annual_inc']

    accepted_train['earliest_cr_yr'] = pd.to_datetime(accepted_train.earliest_cr_line).dt.year
    accepted_train['cr_yr_before_loan'] = accepted_train['issue_yr'] - accepted_train['earliest_cr_yr']
    return accepted_train

In [5]:
accepted_train = Transform_2(accepted_train)
X, y = GetXY(accepted_train, selected_feature)

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [6]:
accepted2017 = accepted2017.loc[accepted2017.loan_status != 'Current']
accepted2017['disposable_inc'] = (1 - accepted2017['dti'])*accepted2017['annual_inc']/12 - accepted2017['installment']
accepted2017['dti_new']= accepted2017['disposable_inc']*12/ accepted2017['annual_inc']

# accepted2017['earliest_cr_yr'] = pd.to_datetime(accepted2017.earliest_cr_line).dt.year
accepted2017['cr_yr_before_loan'] = accepted2017['issue_yr'] - accepted2017['earliest_cr_yr']
#X_test, y_test = GetXY(accepted2017, selected_feature)

X_test, y_test = GetXY(accepted2017, selected_feature)

In [21]:
pd.options.display.float_format = '{:,.2f}'.format
print("Shape of training is", X.shape)
X.head().T

(508676, 61)


,0,1,2,4,5
loan_amnt,15000.000000,10400.000000,21425.000000,7650.000000,9600.000000
int_rate,12.390000,6.990000,15.590000,13.660000,13.660000
installment,336.640000,321.080000,516.360000,260.200000,326.530000
emp_length,10.000000,8.000000,6.000000,0.000000,10.000000
annual_inc,78000.000000,58000.000000,63800.000000,50000.000000,69000.000000
dti,12.030000,14.920000,18.490000,34.810000,25.810000
delinq_2yrs,0.000000,0.000000,0.000000,0.000000,0.000000
fico_range_high,754.000000,714.000000,689.000000,689.000000,684.000000
inq_last_6mths,0.000000,2.000000,0.000000,1.000000,0.000000
open_acc,6.000000,17.000000,10.000000,11.000000,12.000000


Let's test a GBM model on the training data:

In [31]:
gbm = GradientBoostingClassifier(max_depth = 6, n_estimators= 400, max_features = 0.3)
gbm.fit(X, y)
y_score = gbm.predict_proba(X)[:,1]
metrics.roc_auc_score(y, y_score)   # 0.77

0.77623497039370504

In [40]:
y_score_test = gbm.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_score_test)   # 0.69

0.69535824948250335

Let's try a Random Forest Classifier and compare the feature importances between these 2 approaches to zero in on the important features.

In [49]:
rfc = RandomForestClassifier(n_estimators= 600, max_depth = 8)
rfc.fit(X, y)
y_score_rfc = rfc.predict_proba(X)[:,1]
metrics.roc_auc_score(y, y_score_rfc)

0.72440799242850162

In [50]:
y_score_test_rfc = rfc.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test, y_score_test_rfc) 

0.67542684914564854

In [18]:
pd.options.display.float_format = '{:,.2f}'.format
feature_imp = pd.DataFrame({'name': X.columns, 'imp': gbm.feature_importances_}).sort_values(by = 'imp', ascending = False)
feature_imp['mult_gbm'] = feature_imp.imp.max() / feature_imp['imp']

feature_imp['mult_rfc'] = rfc.feature_importances_.max()/rfc.feature_importances_

#rfc_imp = pd.DataFrame({'name': X.columns, 'imp': rfc.feature_importances_}).sort_values(by = 'imp', ascending = False)
#rfc_imp['mult'] = rfc_imp.imp.max() / rfc_imp['imp']
feature_imp


,imp,name,mult_gbm,mult_rfc
52,0.06,install_loan_ratio,1.00,20.31
1,0.06,int_rate,1.01,1.00
55,0.03,install_annual,1.74,27.63
53,0.03,dti_new,1.86,510.86
21,0.03,mo_sin_old_il_acct,1.89,42.07
22,0.03,mo_sin_old_rev_tl_op,1.93,5.92
11,0.03,revol_bal,1.95,334.03
54,0.03,balance_annual_inc,2.02,4.79
51,0.03,disposable_inc,2.05,188.94
44,0.03,tot_hi_cred_lim,2.12,188.88


In [53]:
# Dump all the results
from sklearn.externals import joblib
date_txt = '0402'
joblib.dump(gbm, "gbm_"+ date_txt + '.pkl')
joblib.dump(rfc, "rfc_"+ date_txt + '.pkl')
feature_imp.to_csv("Feature importance 0420.csv", index = False)

# Further testing
Remove some features due to too low values of feature importance

In [18]:
# gbm = joblib.load(r'C:\Users\nkieu\Desktop\Python\Loan data\2018-04-02\gbm_0402.pkl')
# rfc = joblib.load(r'C:\Users\nkieu\Desktop\Python\Loan data\2018-04-02\rfc_0402.pkl')

feature_imp = pd.read_csv(r"C:\Users\nkieu\Desktop\Python\Loan data\2018-04-02\Feature importance 0420.csv")
xx = feature_imp.loc[feature_imp.mult_gbm < 6, ['name','mult_gbm']]
X_reduced = X[xx.name.values] # shape is around 42
X_test_reduced = X_test[xx.name.values]

X_reduced.shape
# gbm_2 = GradientBoostingClassifier(max_depth = 6, n_estimators= 400, max_features = 0.3)
# gbm_2.fit(X_test, y)

# date_txt = '0409'
# joblib.dump(gbm_2, "gbm_" + date_txt + '.pkl')


(508676, 42)

In [17]:
gbm_2 = joblib.load(r'C:\Users\nkieu\Desktop\Python\Loan data\2018-04-09\gbm_0409.pkl')

In [24]:
# Try logistic regression
from sklearn import linear_model
logistic = linear_model.LogisticRegression(penalty='l1', C=0.5)
logistic.fit(X_reduced, y)

joblib.dump(logistic, "logistic"+'0410'+ '.pkl')

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
from sklearn import linear_model

logistic_l2 = linear_model.LogisticRegression(penalty='l2')
logistic_l2.fit(X_reduced, y)
y_score = logistic_l2.predict(X_reduced)
print("In sample", metrics.roc_auc_score(y, y_score)) # 0.51

y_test_score = logistic_l2.predict(X_test_reduced)
print("Out of sample", metrics.roc_auc_score(y_test, y_test_score)) #0.50


In sample 0.511529222645
Out of sample 0.506961365992


In [11]:
logistic = joblib.load(r"C:\Users\nkieu\Desktop\Python\Loan data\logistic0410.pkl")

y_score = logistic.predict(X_reduced)
print("In sample", metrics.roc_auc_score(y, y_score)) # 0.56

y_test_score = logistic.predict(X_test_reduced)
print("Out of sample", metrics.roc_auc_score(y_test, y_test_score)) # 0.55


In sample 0.560843830239
Out of sample 0.555258767477


Let's attempt to overfit the model and see whether there is any room to improve performance on the testing data set.

In [ ]:
### 04-11- increase depth of trees
gbm_4 = GradientBoostingClassifier(max_depth = 10, n_estimators= 200,
                                   subsample = 0.8, max_features = 0.5)
gbm_4.fit(X_reduced, y)
joblib.dump(gbm_4, "gbm4_" + '0411'+ '.pkl')

### Alternatively, RandomUnderSampler
y_score_train = gbm_4.predict_proba(X_reduced)[:,1]
metrics.roc_auc_score(y, y_score_train)   # 0.88

y_score_test = gbm_4.predict_proba(X_test_reduced)[:, 1]
metrics.roc_auc_score(y_test, y_score_test)   # 0.68


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sampler = RandomUnderSampler(random_state=1234)

X_res, y_res = sampler.fit_sample(X_reduced, y)
gbm_5 = GradientBoostingClassifier(max_depth = 6, n_estimators= 200,
                                   subsample = 0.8, max_features = 0.3)
gbm_5.fit(X_res, y_res)
joblib.dump(gbm_5, "gbm5_" + '0411'+ '.pkl')

res = getAUC(gbm_5, X_res, y_res, X_test_reduced, y_test)
# 0.76 in-sample
# 0.86 out of sample

In [ ]:
# Using all samples 
appended = X_reduced.append(X_test_reduced, ignore_index = True)
y_appended = y.append(y_test)

gbm_appended = GradientBoostingClassifier(max_depth = 8, n_estimators= 300,
                                   subsample = 0.8, max_features = 0.3,
                                   learning_rate = 0.8)

gbm_appended.fit(appended, y_appended)
appended_score = gbm_appended.predict_proba(appended)[:, 1]
metrics.roc_auc_score(y_appended, appended_score) # 0.91

joblib.dump(gbm_appended, "gbm_alldata_" + '0411' + '.pkl')

However, if we reduce max_depth = 5, our in sample AUC is reduced to 0.78